In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
# import xlrd
# import openpyxl
pd.set_option('display.max_columns', None)
import numpy as np
from datetime import date, datetime
import time
import os

In [7]:
# driver = webdriver.Chrome(executable_path='/Users/himanshuaggarwal/PycharmProjects/webscraping/chromedriver')
driver = webdriver.Firefox(executable_path='/Users/himanshuaggarwal/PycharmProjects/webscraping/geckodriver')
files = os.listdir()

In [ ]:
files

In [5]:
for file in files:
    if 'Results for UX' in file or 'Results for Data' in file or 'Results for Web Devs' in file:
        os.remove(file)

In [ ]:
driver.find_element

In [8]:
# job_filter = input('enter the job type filter')
# flag = True
# while flag:
#     print('You have entered', job_filter)
#     print('If this is correct press 1')
#     choice = int(input('Enter Choice'))
#     if choice == 1:
#         flag = False
#     else:
#         print('Your Choice was', choice)
#         job_filter = input('enter the job type filter')


def get_data(link):
    # This is to close a pop up window if it appears on the screen
    time.sleep(6)
    try:
        if driver.find_elements_by_class_name('popover-foreground') is not None:
            if len(driver.find_elements_by_class_name('popover-foreground')) > 0:
                driver.find_elements_by_id('popover-link-x')[0].click()
    except:
        pass
    finally:
        html = requests.get(link).content
        soup = BeautifulSoup(html, 'html.parser')
        jobpostings = soup.find_all('div', {'data-tn-component': 'organicJob'})
        title = []
        company = []
        location = []
        summary = []
        salary_snippet = []
        day_posted = []
        job_posting_link = []
        for posting in jobpostings:
            temp = posting.find('a', {'class': 'jobtitle turnstileLink'})['href']
            if posting.find('div', {'class': 'title'}) is not None and len(re.findall('jk.*', temp)) > 0:
                title.append(posting.find('div', {'class': 'title'}).text.strip())
                if posting.find('div', {'class': 'sjcl'}) is not None:
                    temp = posting.find('div', {'class': 'sjcl'}).text.strip().split('\n')
                    sjcl = []
                    for item in temp:
                        if item != '':
                            sjcl.append(item)
                    company.append(sjcl[0])
                    location.append(sjcl[-1])
                else:
                    company.append('NA')
                    location.append('NA')
                if posting.find('a', {'class':'jobtitle turnstileLink'}) is not None:
                    temp = posting.find('a', {'class':'jobtitle turnstileLink'})['href']
                    try:
                        temp = 'https://www.indeed.com/viewjob?' + re.findall('jk.*', temp)[0]
                        job_posting_link.append(temp)
                    except:
                        pass
                else:
                    job_posting_link.append('NA')
                if posting.find('div', {'class': 'summary'}) is not None:
                    summary.append(posting.find('div', {'class': 'summary'}).text.strip())
                else:
                    summary.append('NA')
                if posting.find('div', {'class': 'salarysnippet'}) is not None:
                    salary_snippet.append(posting.find('div', {'class': 'salarysnippet'}))
                else:
                    salary_snippet.append('NA')
                if posting.find('div', {'class': 'jobsearch-SerpJobCard-footerActions'}) is not None:
                    day_posted.append(
                        posting.find('div', {'class': 'jobsearch-SerpJobCard-footerActions'}).text.strip().split(' -')[
                            0].strip())
                else:
                    day_posted.append('NA')
        temp_df = pd.DataFrame()
        temp_df['Title'] = title
        temp_df['Company'] = company
        temp_df['Summary'] = summary
        temp_df['Location'] = location
        temp_df['DayPosted'] = day_posted
        temp_df['JobPostingLink'] = job_posting_link
    return temp_df


jobs = ['Data Analysis', 'Data Analyst', 'Developer', 'Software Engineer', 'UX', 'Product Designer', 'UX/UI']
# jobs = ['UX']
for job in jobs:
    driver.get('https://www.indeed.com')
    time.sleep(2)
    try:
        driver.find_elements_by_class_name('icl-LegalConsentBanner-action')[0].click()
    except:
        pass
    # Deleting anything that might be there in the cell and input new values for Job Type
    driver.find_element_by_id('text-input-what').send_keys(Keys.COMMAND + "a")
    driver.find_element_by_id('text-input-what').send_keys(Keys.DELETE)
    driver.find_elements_by_id('text-input-what')[0].send_keys(job)
    time.sleep(5)
    # location = input('enter the location filter')
    # flag = True
    # while flag:
    #     print('You have entered', location)
    #     print('If this is correct press 1')
    #     choice = int(input('Enter Choice'))
    #     if choice == 1:
    #         flag = False
    #     else:
    #         print('Your Choice was', choice)
    #         location = input('enter the location filter')

    # Deleting anything that might be there in the cell and input new values for Location
    driver.find_element_by_id('text-input-where').send_keys(Keys.COMMAND + "a")
    driver.find_element_by_id('text-input-where').send_keys(Keys.DELETE)
    driver.find_elements_by_id('text-input-where')[0].send_keys('Miami, FL')
    time.sleep(3)
    # Since the clicks was not working. We used the return key in the same cell
    # driver.find_elements_by_class_name('icl-WhatWhere-buttonWrapper')[0].click()
    driver.find_elements_by_id('text-input-where')[0].send_keys(Keys.RETURN)
    time.sleep(3)
    try:
        driver.find_elements_by_partial_link_text('Advanced Job Search')[0].click()
        time.sleep(2)
        try:
            driver.find_elements_by_class_name('icl-LegalConsentBanner-action')[0].click()
        except:
            pass
        select = Select(driver.find_element_by_id('radius'))
        # select by visible text
        # select.select_by_visible_text('within 50 miles of')
        # select by value
        select.select_by_value('50')
        select = Select(driver.find_element_by_id('fromage'))
        select.select_by_value('7')
        select = Select(driver.find_element_by_id('limit'))
        select.select_by_value('50')
        time.sleep(2)
        driver.find_element_by_id('fj').click()
    except:
        driver.find_element_by_id('fj').click()
    data = pd.DataFrame(columns=['Title', 'Company', 'Summary', 'Location', 'DayPosted', 'JobPostingLink'])
    time.sleep(3)
    try:
        if driver.find_elements_by_class_name('popover-foreground') is not None:
            if len(driver.find_elements_by_class_name('popover-foreground')) > 0:
                driver.find_elements_by_id('popover-link-x')[0].click()
                print('PopOver Eliminated')
                time.sleep(4)
    finally:
        print('Starting Scraping')

    FLAG = True
    counter = 1
    while FLAG:
        try:
            if driver.find_elements_by_class_name('popover-foreground') is not None:
                if len(driver.find_elements_by_class_name('popover-foreground')) > 0:
                    driver.find_elements_by_id('popover-link-x')[0].click()
                    print('PopOver Eliminated')
                    time.sleep(4)
        except:
            pass
        if driver.current_url is not None:
            print(driver.current_url)
            print(counter)
            counter += 1
            # Go to the next page. We are using find_element instead of elements so that we only go to the next page
            url = driver.current_url
            scraped_page_df = get_data(url)
            data = pd.concat([data, scraped_page_df], axis=0)
            try:
                time.sleep(3)
                driver.find_element_by_class_name('pn').click()
            except:
                print('Next page was not found or click did not word')
                FLAG = False
        else:
            FLAG = False
            print('Exit - url not found')
        if data.shape[0] > 250:
            FLAG = False
            print('Exit - Got 100 rows')

    data['Cohort'] = job
    data['Date'] = date.today()
    data['Date & Time'] = datetime.now()
    data['DayPosted'] = list(map(lambda x: x[0:11], data['DayPosted']))
    data = data.reset_index()
    print(data)
    # Saving to a csv file

    if 'ux' in job.lower():
        if '/' in job:
            data.to_excel('Results for UXUI.xlsx', sheet_name='Sheet')
        else:
            data.to_excel('Results for UX.xlsx', sheet_name=job)
    elif 'designer' in job.lower():
        data.to_excel('Results for Product Designer.xlsx', sheet_name=job)
    elif 'analysis' in job.lower():
        data.to_excel('Results for Data Analysis.xlsx', sheet_name=job)
    elif 'analyst' in job.lower():
        data.to_excel('Results for Software Engineer.xlsx', sheet_name=job)
        data.to_excel('Results for Data Analyst.xlsx', sheet_name=job)
    elif 'developer' in job.lower():
        data.to_excel('Results for Web Developers.xlsx', sheet_name=job)
    elif 'software' in job.lower():
        pass
try:
    df1 = pd.read_excel('Results for Data Analysis.xlsx')
    df2 = pd.read_excel('Results for Data Analyst.xlsx')
    df = pd.concat([df1, df2], axis=0)
    df = df.drop_duplicates(subset=['JobPostingLink']).reset_index(drop=True)
    df = df.drop(columns =['Date', 'index'], axis=1)
    df.to_excel('DATA_final.xlsx', sheet_name='DataCohortSheet')
    os.remove('Results for Data Analysis.xlsx')
    os.remove('Results for Data Analyst.xlsx')
except:
    pass

try:
    df1 = pd.read_excel('Results for UXUI.xlsx')
    df2 = pd.read_excel('Results for UX.xlsx')
    df = pd.concat([df1, df2], axis=0)
    df = df.drop(columns=['Unnamed: 0', 'Date', 'index'], axis=1)
    df = df.drop_duplicates(subset=['JobPostingLink']).reset_index(drop=True)
    df.to_excel('UX_final.xlsx', sheet_name='UXCohortSheet')
except:
    pass
files = os.listdir()
for file in files:
    if ('Results for UX' in file) or ('Results for Data' in file):
        os.remove(file)

PopOver Eliminated
Starting Scraping
https://www.indeed.com/jobs?as_and=Data+Analysis&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=50&l=Miami%2C+FL&fromage=7&limit=50&sort=&psf=advsrch&from=advancedsearch#
1
https://www.indeed.com/jobs?q=Data+Analysis&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
2
https://www.indeed.com/jobs?q=Data+Analysis&l=Miami%2C+FL&radius=50&limit=50&fromage=7
3
https://www.indeed.com/jobs?q=Data+Analysis&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
4
https://www.indeed.com/jobs?q=Data+Analysis&l=Miami%2C+FL&radius=50&limit=50&fromage=7
5
https://www.indeed.com/jobs?q=Data+Analysis&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
6
https://www.indeed.com/jobs?q=Data+Analysis&l=Miami%2C+FL&radius=50&limit=50&fromage=7
7
https://www.indeed.com/jobs?q=Data+Analysis&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
8
Exit - Got 100 rows
     index                                              Title  \
0        0   Tempor

Starting Scraping
https://www.indeed.com/jobs?as_and=Data+Analyst&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=50&l=Miami%2C+FL&fromage=7&limit=50&sort=&psf=advsrch&from=advancedsearch
1
https://www.indeed.com/jobs?q=Data+Analyst&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
2
https://www.indeed.com/jobs?q=Data+Analyst&l=Miami%2C+FL&radius=50&limit=50&fromage=7
3
https://www.indeed.com/jobs?q=Data+Analyst&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
4
https://www.indeed.com/jobs?q=Data+Analyst&l=Miami%2C+FL&radius=50&limit=50&fromage=7
5
https://www.indeed.com/jobs?q=Data+Analyst&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
6
https://www.indeed.com/jobs?q=Data+Analyst&l=Miami%2C+FL&radius=50&limit=50&fromage=7
7
https://www.indeed.com/jobs?q=Data+Analyst&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
8
https://www.indeed.com/jobs?q=Data+Analyst&l=Miami%2C+FL&radius=50&limit=50&fromage=7
9
https://www.indeed.com/jobs?q=Data+Analyst&l

Starting Scraping
https://www.indeed.com/jobs?as_and=Developer&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=50&l=Miami%2C+FL&fromage=7&limit=50&sort=&psf=advsrch&from=advancedsearch
1
https://www.indeed.com/jobs?q=Developer&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
2
https://www.indeed.com/jobs?q=Developer&l=Miami%2C+FL&radius=50&limit=50&fromage=7
3
https://www.indeed.com/jobs?q=Developer&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
4
https://www.indeed.com/jobs?q=Developer&l=Miami%2C+FL&radius=50&limit=50&fromage=7
5
https://www.indeed.com/jobs?q=Developer&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
6
https://www.indeed.com/jobs?q=Developer&l=Miami%2C+FL&radius=50&limit=50&fromage=7
7
https://www.indeed.com/jobs?q=Developer&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
8
https://www.indeed.com/jobs?q=Developer&l=Miami%2C+FL&radius=50&limit=50&fromage=7
9
https://www.indeed.com/jobs?q=Developer&l=Miami%2C+FL&radius=50&limit=5

Starting Scraping
https://www.indeed.com/jobs?as_and=Software+Engineer&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=50&l=Miami%2C+FL&fromage=7&limit=50&sort=&psf=advsrch&from=advancedsearch
1
https://www.indeed.com/jobs?q=Software+Engineer&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
2
https://www.indeed.com/jobs?q=Software+Engineer&l=Miami%2C+FL&radius=50&limit=50&fromage=7
3
https://www.indeed.com/jobs?q=Software+Engineer&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
4
https://www.indeed.com/jobs?q=Software+Engineer&l=Miami%2C+FL&radius=50&limit=50&fromage=7
5
https://www.indeed.com/jobs?q=Software+Engineer&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
6
https://www.indeed.com/jobs?q=Software+Engineer&l=Miami%2C+FL&radius=50&limit=50&fromage=7
7
https://www.indeed.com/jobs?q=Software+Engineer&l=Miami%2C+FL&radius=50&limit=50&fromage=7&start=50
8
https://www.indeed.com/jobs?q=Software+Engineer&l=Miami%2C+FL&radius=50&limit=50&fromage=7
9

Starting Scraping
https://www.indeed.com/jobs?as_and=UX&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=50&l=Miami%2C+FL&fromage=7&limit=50&sort=&psf=advsrch&from=advancedsearch
1
Next page was not found or click did not word
    index                                              Title  \
0       0               Graphic Designer (Miami) (Part-Time)   
1       1                                      Designer - SG   
2       2                   Principal Human Vision Scientist   
3       3                   Designed Focused UI/UX Developer   
4       4                                     Sr. UX Manager   
5       5                      Senior Human Vision Scientist   
6       6                          Associate Product Manager   
7       7                   Senior Product Manager - Quantum   
8       8                         US-Technology Consultant-2   
9       9                   Associate Integration Specialist   
10     10  eCommerce Photography Technician 

Starting Scraping
https://www.indeed.com/jobs?as_and=UX%2FUI&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=50&l=Miami%2C+FL&fromage=7&limit=50&sort=&psf=advsrch&from=advancedsearch
1
Next page was not found or click did not word
   index                                              Title  \
0      0               Graphic Designer (Miami) (Part-Time)   
1      1                   Principal Human Vision Scientist   
2      2                      Senior Human Vision Scientist   
3      3  eCommerce Photography Technician & Graphic Des...   
4      4                                            UX Lead   

                                    Company  \
0                                  Cozymeal   
1                          Magic Leap, Inc.   
2                          Magic Leap, Inc.   
3  Hard Rock Cafe International (USA), Inc.   
4                  Cross Country Healthcare   

                                             Summary  \
0  Experience with UX and

In [ ]:
# x = [5,10,4]#200
# temp = 0
# for i in range(x.pop()):
#     for loop in range(i):
#         temp = temp + x[i]
#     x[i] = temp
# temp        

In [ ]:
import sympy 
x = [1,3,5]

In [ ]:
def multiplyArray(x):
    temp1=0
    for i in range(1,len(x)):
        for num in range(x[i]):
            temp1 = temp1 + x[i-1]
        x[i] = temp1
        if i!=len(x)-1:
            temp1=0
    return(temp1)


In [ ]:
for i in range(10):
    num = multiplyArray(x) + 1
    if sympy.isprime(num)==False:
        primes=[i for i in range(2,num) if sympy.isprime(i)==True]
        x = primes

In [ ]:
x